In [2]:
!pip install psycopg2
!pip install pandas
!pip install sqlalchemy


In [18]:
import uuid

In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
db_host = "db-application-process-dataeng.postgres.database.azure.com"
db_port = "5432"
db_name = "candidate_8qls"
db_user = "salma_lahrech"
db_password = "@GxxPNkLd3rrqi3S"

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    port=db_port,
    dbname=db_name,
    user=db_user,
    password=db_password
)

cur = conn.cursor()


#load data into dataframe
df_treecsv = pd.read_csv('data/tree_data.csv')
# Create an SQLAlchemy engine
connection_string = 'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)
# Append data from DataFrame to the PostgreSQL table
df_treecsv.to_sql('raw_data', engine, index=False, if_exists='append', method='multi', chunksize=1000)
engine.dispose()

In [ ]:

#putting the whole table in data frame
cur.execute("SELECT * FROM raw_data")
results = cur.fetchall()
df_rawdata = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])


**<span style="color:red;">Please remember to respond to the questions in the readme.md file as well and good luck :)</span>**

# DATA QUALITY

### Question 1
In the dataset provided to you, the focus is on trees, farmers, and parcels. At PUR, our primary activity involves planting trees on parcels owned by farmers. This gives you an opportunity to examine data similar to what you would encounter if you eventually join us.

How can you evaluate the quality of the data you have received ? Provide a quick analysis using the tool(s) / libraries of your choice.

In [14]:
df_rawdata.head()

,longitude,latitude,size_height,size_width,tree_species,farmer_first_name,farmer_last_name,farmer_national_id,parcel_id,parcel_area,created_at,farmer_birthday,species_latin_name
0,-70.749857,-11.464434,15.49,91.63,Maple,Alejandro,Lopez,4518066197634485419,13f5fcaa-6a0f-5fea-b1b1-7de916e89f62,191.51,2021-08-27 11:32:40,None,Acer
1,-70.529544,-11.494757,21.87,107.71,Maple,Rosa,Rodriguez,4590364070282946098,64f69e1e-8c2a-57f9-bce5-ac10ef1f61b0,43.96,2021-08-23 20:21:20,None,Acer
2,-70.610692,-11.338272,7.06,24.29,Fir,Sebastian,Gonzalez,4869115316512327290,421ddcac-3622-5a93-9500-e30551abab45,164.11,2021-09-07 05:31:55,None,Abies
3,-70.591786,-11.334598,13.44,49.69,Eucalyptus,Sebastian,Gonzalez,4869115316512327290,421ddcac-3622-5a93-9500-e30551abab45,164.11,2021-08-15 10:11:03,None,Eucalyptus
4,-70.550708,-11.472554,20.12,None,Willow,Fernando,Gonzalez,73514415500322874,1005a79a-4349-57a0-af7f-9908d7645291,90.15,2021-09-04 01:59:58,None,Salix


In [15]:
df_rawdata.describe(include='all')

,longitude,latitude,size_height,size_width,tree_species,farmer_first_name,farmer_last_name,farmer_national_id,parcel_id,parcel_area,created_at,farmer_birthday,species_latin_name
count,1358,1358,1358,785,1358,1358,1358,1358,1358,1358,1358,331,1358
unique,320,320,1098,755,9,10,6,11,11,20,1358,10,9
top,-70.749857,-11.464434,14.55,115.55,Bamboo,Carlos,Rodriguez,2525770196998878965,194c0d73-0092-5c49-9a1a-28b9a2c3ac41,102.38,2021-08-27 11:32:40,1970-08-08,Bambusoideae
freq,5,5,5,3,187,192,471,192,192,183,1,62,187


In [20]:
df_rawdata['TreeID'] = [uuid.uuid4() for _ in range(len(df_rawdata))]
df_rawdata.head()

,longitude,latitude,size_height,size_width,tree_species,farmer_first_name,farmer_last_name,farmer_national_id,parcel_id,parcel_area,created_at,farmer_birthday,species_latin_name,TreeID
0,-70.749857,-11.464434,15.49,91.63,Maple,Alejandro,Lopez,4518066197634485419,13f5fcaa-6a0f-5fea-b1b1-7de916e89f62,191.51,2021-08-27 11:32:40,None,Acer,6a38783c-dbf3-43a9-b3d3-7b22a73d2f5e
1,-70.529544,-11.494757,21.87,107.71,Maple,Rosa,Rodriguez,4590364070282946098,64f69e1e-8c2a-57f9-bce5-ac10ef1f61b0,43.96,2021-08-23 20:21:20,None,Acer,d40f7733-fdc5-4fca-ac1b-149495e458b9
2,-70.610692,-11.338272,7.06,24.29,Fir,Sebastian,Gonzalez,4869115316512327290,421ddcac-3622-5a93-9500-e30551abab45,164.11,2021-09-07 05:31:55,None,Abies,4ce8dbd5-6732-43b6-ad96-85c2e52be02d
3,-70.591786,-11.334598,13.44,49.69,Eucalyptus,Sebastian,Gonzalez,4869115316512327290,421ddcac-3622-5a93-9500-e30551abab45,164.11,2021-08-15 10:11:03,None,Eucalyptus,c1961ff8-0be8-449a-b301-1034374dfad1
4,-70.550708,-11.472554,20.12,None,Willow,Fernando,Gonzalez,73514415500322874,1005a79a-4349-57a0-af7f-9908d7645291,90.15,2021-09-04 01:59:58,None,Salix,1e85a703-48e4-4286-aefb-d3cfea9e7934


In [22]:
#splitting the dataframe to 3 dataframes
farmers_df = df_rawdata[['farmer_national_id', 'farmer_first_name', 'farmer_last_name', 'farmer_birthday']].drop_duplicates()
parcels_df = df_rawdata[['parcel_id', 'farmer_national_id', 'parcel_area','longitude','latitude']].drop_duplicates()
trees_df = df_rawdata[['TreeID', 'parcel_id', 'species_latin_name', 'size_height', 'size_width', 'created_at']].drop_duplicates()

In [23]:
#By using Python, provide a modelization of a farmer and by running your code,
# you would get a farmer's instance (the data can be mocked or from the database)
class Farmer:
    def __init__(self, first_name, last_name, national_id, birthday, address):
        self.first_name = first_name
        self.last_name = last_name
        self.national_id = national_id
        self.birthday = birthday
        self.address = address

    def __repr__(self):
        return (f"Farmer({self.first_name} {self.last_name}, "
                f"ID: {self.national_id}, Birthday: {self.birthday}, Address: {self.address})")

# Create an instance of Farmer with mock data
mock_farmer = Farmer("John", "Doe", "123456789", "1980-07-01", "123 Elm Street")

# Print the farmer's instance to see the output
print(mock_farmer)


Farmer(John Doe, ID: 123456789, Birthday: 1980-07-01, Address: 123 Elm Street)


### Question 2

One of the column is provided with sparse information.

What could be a strategy to handle this situation ? Implement it here (you do not have to persist this solution in the database).

In [ ]:
#delete the whole column 

# DATA ANALYSIS

#### Question 3
The marketing department seeks information on the total count of farmers and parcels participating in this project.

What output could you provide them to give the best insight about this problematic ?

#### Question 4
One of our agroforestry expert is wondering if there's a correlation between the weidth and the height of a tree.

Can you provide an answer to her?

In [29]:
!pip install scipy
from scipy.stats import pearsonr

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.6 kB 330.3 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/60.6 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 457.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/45.9 MB ? eta -:--:--
   ---------------------------------------- 0.4/45.9 MB 8.9 MB/s eta 0:00:06
   - -------------------------------------- 1.6/45.9 MB 16.8 MB/s eta 0:00:03
   -- ------------------------------------- 2.7/45.9 MB 19.4 MB/s eta 0:00:03
   --- ------------------------------------ 3.9/45.9 MB 20.7 MB/s eta 0:00:03
   ---- ----------------------------------- 5.1/45.9 MB 21.7 MB/s eta 0:00:02
   ----- ---------------------------------- 6.3/45.9 MB 21.1 MB/s eta 0:00:02
   ------ --------------------------------- 7.5/45.9 MB 21.7 MB/s eta 0:00:02
   ------- -------------------------------- 8.7/45.9 MB 22.3 MB/s eta 0:0

In [27]:
# Check for any missing values in the relevant columns
print(df_rawdata[['size_width', 'size_height']].isnull().sum())
df_corelation=df_rawdata.dropna(subset=['size_width', 'size_height'])
df_corelation.head()


# Calculate the Pearson correlation coefficient
correlation, _ = pearsonr(df_corelation['size_width'], df_corelation['size_height'])
print(f'The Pearson correlation coefficient between tree width and height is: {correlation}')


size_width     573
size_height      0
dtype: int64


,longitude,latitude,size_height,size_width,tree_species,farmer_first_name,farmer_last_name,farmer_national_id,parcel_id,parcel_area,created_at,farmer_birthday,species_latin_name,TreeID
0,-70.749857,-11.464434,15.49,91.63,Maple,Alejandro,Lopez,4518066197634485419,13f5fcaa-6a0f-5fea-b1b1-7de916e89f62,191.51,2021-08-27 11:32:40,None,Acer,6a38783c-dbf3-43a9-b3d3-7b22a73d2f5e
1,-70.529544,-11.494757,21.87,107.71,Maple,Rosa,Rodriguez,4590364070282946098,64f69e1e-8c2a-57f9-bce5-ac10ef1f61b0,43.96,2021-08-23 20:21:20,None,Acer,d40f7733-fdc5-4fca-ac1b-149495e458b9
2,-70.610692,-11.338272,7.06,24.29,Fir,Sebastian,Gonzalez,4869115316512327290,421ddcac-3622-5a93-9500-e30551abab45,164.11,2021-09-07 05:31:55,None,Abies,4ce8dbd5-6732-43b6-ad96-85c2e52be02d
3,-70.591786,-11.334598,13.44,49.69,Eucalyptus,Sebastian,Gonzalez,4869115316512327290,421ddcac-3622-5a93-9500-e30551abab45,164.11,2021-08-15 10:11:03,None,Eucalyptus,c1961ff8-0be8-449a-b301-1034374dfad1
5,-70.724495,-11.415593,8.75,34.79,Cedar,Rosa,Rodriguez,4590364070282946098,64f69e1e-8c2a-57f9-bce5-ac10ef1f61b0,43.96,2021-09-05 13:12:18,None,Cedrus,cc25ac4c-8e98-4f69-9197-469af0376f88


#### Question 5
At PUR, our mission is to accelerate biodiversity regeneration through strategic tree planting. A key factor in this effort is identifying which tree species exhibit the most significant growth over time, as this can influence our planting decisions.

Therefore, we're interested in understanding the growth patterns of these species. Could you analyze this data to identify which species demonstrates the highest growth rate? For the purpose of this analysis, consider that a species is considered to be growing if its height and/or width increases over time.

Please provide an analysis that ranks the tree species by their growth rate, helping us to pinpoint the species that could contribute most effectively to our biodiversity regeneration efforts

### Question 6 (Bonus) - ONLY IF YOU APPLIED TO THE DATA/BI ENGINEERING POSITION

The client who is financing this agroforestry project is asking you to display a map where he could:
- Vizualize **the location** of the planted trees
- Distinguish the trees by **height**


Can you create such visualization using the tool(s) / libraries of your choice.

### Question 6 (Bonus) - ONLY IF YOU APPLIED TO THE CONFIRMED DATA-ENGINEERING POSITION

Using Python and any framework of your choice, can you create an endpoint that allows us to query the number of parcels associated with a specific farmer.

We expect a response that looks like below but feel free to add any relevant information in this payload. For your information, the response_id is a randomly generated UUID.

In [ ]:
{
    "parcels": [
        {id: "4f2cd9d6-5db6-5081-92a5-517f1b92949a"},
        {id: "1005a79a-4349-57a0-af7f-9908d7645291"}
    ],
    "reponse_id": "0cc12fda-deca-4f24-918b-f0884d2bb910"
    "total": 2
}

In [ ]:
cur.close()
conn.close()